In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json

In [2]:
df = pd.read_csv('data/US_Accidents_Dec19.csv',encoding='utf-8')
df.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


In [3]:
df.columns

Index(['ID', 'Source', 'TMC', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)',
       'Description', 'Number', 'Street', 'Side', 'City', 'County', 'State',
       'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [4]:
columns_of_interest=['ID','Severity','Start_Time','End_Time','Start_Lat', 'Start_Lng', 'City', 'County', 'State',
       'Zipcode', 'Country','Visibility(mi)', 'Weather_Condition','Precipitation(in)','Sunrise_Sunset','Civil_Twilight']
condition=df['State'] =='TX'
txDF=df[condition][columns_of_interest]

txDF.head()

txDF['City'].value_counts()

Houston                    93245
Austin                     58553
Dallas                     57823
San Antonio                21613
El Paso                     9352
Fort Worth                  8746
Arlington                   3011
Irving                      2697
Mesquite                    2180
Grand Prairie               2159
Pflugerville                2152
Manor                       1941
Del Valle                   1451
Spring                      1398
Humble                      1274
Plano                       1222
Richardson                  1138
Carrollton                  1089
Garland                      989
Wichita Falls                819
Pasadena                     718
Lewisville                   691
Kingwood                     688
Round Rock                   671
Denton                       603
Hurst                        576
Lancaster                    573
Duncanville                  532
Grapevine                    526
Bellaire                     492
          

In [6]:
# Number of accidents that happen at Night
txDF.groupby(['Sunrise_Sunset'])['ID'].count()

Sunrise_Sunset
Day      230645
Night     67417
Name: ID, dtype: int64

In [23]:
print(len(txDF['Precipitation(in)']))
#print(len(txDF['Precipitation(in)'].dropna()))
#precipitation=txDF['Precipitation(in)'].dropna()

condition = txDF['Precipitation(in)'].notnull() & txDF['Precipitation(in)'] > 0.0
precipitation= txDF[condition]

print(len(precipitation))
print(precipitation.head())

precipitation['Precipitation(in)'].describe()

# Over 75% where there is some precipitation happen with less than 0.1 inches of rain

#precipitation.describe()


298062
14434
              ID  Severity           Start_Time             End_Time  \
261605  A-261607         2  2016-12-02 13:37:39  2016-12-02 14:50:57   
261606  A-261608         2  2016-12-02 13:38:31  2016-12-02 14:52:57   
261607  A-261609         3  2016-12-02 13:40:53  2016-12-02 14:55:53   
261610  A-261612         2  2016-12-02 13:59:51  2016-12-02 15:29:39   
261615  A-261617         2  2016-12-02 14:25:24  2016-12-02 15:40:08   

        Start_Lat  Start_Lng    City  County State     Zipcode Country  \
261605  30.230623 -97.813980  Austin  Travis    TX  78745-1331      US   
261606  30.334192 -97.718262  Austin  Travis    TX  78752-3510      US   
261607  30.231228 -97.742569  Austin  Travis    TX       78741      US   
261610  30.257950 -97.715096  Austin  Travis    TX  78702-4759      US   
261615  30.233406 -97.795509  Austin  Travis    TX       78704      US   

        Visibility(mi) Weather_Condition  Precipitation(in) Sunrise_Sunset  \
261605            10.0        L

count    14434.000000
mean         0.099047
std          0.184035
min          0.010000
25%          0.010000
50%          0.040000
75%          0.100000
max          2.940000
Name: Precipitation(in), dtype: float64

In [24]:
txDF['Weather_Condition'].value_counts()

Clear                                  76753
Mostly Cloudy                          47948
Overcast                               45135
Partly Cloudy                          37059
Scattered Clouds                       25356
Fair                                   24646
Light Rain                             10485
Cloudy                                  9163
Rain                                    2457
Fog                                     2303
Haze                                    2189
Light Drizzle                           1321
Heavy Rain                               900
Light Thunderstorms and Rain             831
Thunderstorm                             586
Thunderstorms and Rain                   395
Heavy Thunderstorms and Rain             373
Light Rain with Thunder                  356
Thunder                                  256
Mostly Cloudy / Windy                    253
T-Storm                                  250
Fair / Windy                             241
Mist      

In [25]:
txDF['Visibility(mi)'].value_counts()

10.00     232214
9.00       10617
7.00        8271
8.00        6966
5.00        5329
6.00        4797
4.00        4046
3.00        3943
13.00       3760
2.00        2511
2.50        1629
1.00        1444
0.20        1379
0.50         912
1.50         855
1.80         697
0.80         627
1.20         486
0.10         180
11.00        146
12.00        143
20.00        122
0.75          97
0.25          84
0.00          79
0.40          19
2.80          14
0.70          12
0.12          11
1.90           8
15.00          7
1.60           5
5.50           5
0.06           5
2.20           4
10.50          3
111.00         3
0.60           3
3.50           3
1.10           2
0.90           2
0.63           2
46.00          1
1.40           1
Name: Visibility(mi), dtype: int64